In [1]:
!pip install konlpy

In [1]:
import pandas as pd
import re
from konlpy.tag import Okt
from transformers import BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, AutoModel
from safetensors.torch import load_file
from sklearn.metrics.pairwise import cosine_similarity
import torch
import os

In [3]:
df = pd.read_csv('law_data_3.csv')
df1 = pd.read_csv('law_data_2.csv')
df2 = pd.read_csv('law_data_1.csv')

In [4]:
df = pd.concat([df1, df2], ignore_index=True)

In [5]:
df['전문'] = df['전문'].fillna('')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta-multitask")
model = AutoModel.from_pretrained("BM-K/KoSimCSE-roberta-multitask")

tokenizer_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

C:\Users\KOSA\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\KOSA\.cache\huggingface\hub\models--BM-K--KoSimCSE-roberta-multitask. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [8]:
okt = Okt()

In [9]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    text = text.lower()
    text = ' '.join(okt.morphs(text))
    return text

In [10]:
def chunk_text(text, chunk_size=128, overlap=32):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
        if i + chunk_size >= len(words):
            break
    return chunks

In [11]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return cls_embedding

In [ ]:
df['processed_text'] = df['전문'].apply(preprocess_text)
df['chunks'] = df['processed_text'].apply(lambda x: chunk_text(x))

In [ ]:
chunk_embeddings = []
chunk_to_doc = []

In [ ]:
for idx, chunks in enumerate(df['chunks']):
    for chunk in chunks:
        chunk_embedding = encode_text(chunk)
        chunk_embeddings.append(chunk_embedding)
        chunk_to_doc.append(idx)

In [ ]:
chunk_embeddings = torch.stack(chunk_embeddings)

In [ ]:
# save_path = '/content/drive/My Drive/content/chunk_embeddings.pt'
# torch.save({
#     'chunk_embeddings': chunk_embeddings,
#     'chunk_to_doc': chunk_to_doc
# }, save_path)

In [12]:
save_path = 'chunk_embeddings.pt'
checkpoint = torch.load(save_path)
chunk_embeddings = checkpoint['chunk_embeddings']
chunk_to_doc = checkpoint['chunk_to_doc']

In [13]:
  def search_query(query, df, chunk_embeddings, chunk_to_doc, top_k=5):
    query = preprocess_text(query)
    query_embedding = encode_text(query).unsqueeze(0).to(device)

    query_embedding_cpu = query_embedding.cpu()
    chunk_embeddings_cpu = chunk_embeddings.cpu()

    similarities = cosine_similarity(query_embedding_cpu, chunk_embeddings_cpu).flatten()

    top_indices = similarities.argsort()[-top_k:][::-1]

    search_results = df.iloc[[chunk_to_doc[i] for i in top_indices]]
    return search_results[['주문']]

In [14]:
query = "내에서 피해자 D(남, 63세)과 함께 술을 마시던 중 욕설 등 말다툼 중에 화가 나 그곳에 있던 위험한 물건인 과도(칼날 길이 약 9cm, 전체 길이 약 19.5cm)를 집어 들고 피해자의 좌측 가슴 부위를 1회 찔러 피해자에게 약 2주간의 치료를 요하는 흉부의 상세불명 부분의 열린상처 등 상해를 가하였다"
#query = '고양시 일산동구 B에 있는 C 1층에 있는 피해자 D가 운영하는 E에서, 계산대에 올려 놓은 우유 2팩을 바닥으로 던져 우유가 쏟아지게 하고, 소리를 지르는 등 소란을 피워 위력으로써 피해자의 제과점 운영 업무를 방해하였다'
#query = '음주운전을 하여 행인 2명을 부상시키고 후속 처리를 하지 않고 도망쳤다. 동일 전과가 이전에 있다'
#query = '보일러의 설치상 하자 등을 발견할 것을 기대할 수도 없다'
#query = ' 이러한 경우 자동차의 운전업무에 종사하는 사람에게는 전방 교통상황을 잘 보고 조향 및 제동 장치 등을 정확히 조작하는 등 안전하게 운전하여 사고를 미리 방지하여야 할업무상 주의의무가 있다'
search_result = search_query(query, df, chunk_embeddings, chunk_to_doc)

In [15]:
print(search_result)

                                                     주문
1840  피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 3년간 위 형의 집행을 유...
1499  피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유...
1402  피고인을 징역 6월에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유...
1848  피고인을 징역 1년에 처한다. 다만, 이 판결 확정일부터 2년간 위 형의 집행을 유...
1145           피고인을 징역 6년에 처한다. 압수된 식칼 1개(증 제1호)를 몰수한다.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
gen_model_path = 'beomi-Llama-3-KoEn-8B-Instruct-preview'
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_path)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
